In [1]:
!pip install jellyfish
!pip install faker

In [1]:
import pyspark
import jellyfish
import pandas as pd
import numpy as np
from typing import List
import os
import math
import time
from collections import defaultdict
from packages.generateDataSets import SyntheticMatcherDataset
from packages.calculateStatistics import DatasetEvaluator
from itertools import combinations

In [2]:

# Data for df1
data1 = [
    ['ID00005', 'N039', 'E298', 'Q412', 'V409', 'R232'], #TP1
    ['ID00009', 'R822', 'W179', 'H017', 'P323', 'F298'], #TP2
    ['ID00007', 'R449', 'X716', 'M948', 'G667', 'S702'], #TP3
    ['ID00004', 'N002', 'E396', 'N843', 'I458', 'S719'], #TP4
    ['ID10004', 'N002', 'E396', 'N853', 'I623', 'S569'], #FN1
    ['NEW72378', 'J547', 'B222', 'G492', 'R551', 'S490'], #FP1
    ['ID00008', 'N322', 'K685', 'T442', 'C825', 'W967'], #FP2
    ['ID00000', 'W815', 'L281', 'R155', 'F768', 'B914'],
    ['ID00001', 'C172', 'B326', 'X400', 'M508', 'O776'],
    ['ID00002', 'V683', 'C265', 'J127', 'D589', 'F482'],
    ['ID00003', 'E851', 'P721', 'F745', 'D863', 'K229'],
    ['ID00016', 'T873', 'D670', 'U046', 'Z181', 'X621'],
    ['ID00017', 'F327', 'G856', 'E567', 'O929', 'Q721'],
    ['ID00010', 'O283', 'T723', 'Z034', 'V319', 'X338'],
]

# Data for df2
data2 = [
    ['ID00005', 'R746', 'E298', 'Q412', 'L291', 'R232'], #TP1
    ['ID00009', 'R822', 'W179', 'H017', 'P323', 'F298'], #TP2
    ['ID00007', 'Z011', 'X716', 'M948', 'W967', 'S702'], #TP3
    ['ID00004', 'N002', 'E396', 'N843', 'V935', 'S719'], #TP4
    ['ID10004', 'N002', 'E396', 'N553', 'I453', 'S459'], #FN1
    ['NEW80187', 'J547', 'B222', 'G492', 'W673', 'S490'], #FP1
    ['NEW30110', 'N322', 'K685', 'T432', 'C225', 'W967'], #FP2
    ['NEW72832', 'F875', 'Q768', 'H822', 'Z154', 'X678'], 
    ['NEW30110', 'R560', 'C434', 'M687', 'Q689', 'Q863'],
    ['NEW81243', 'R762', 'N687', 'A109', 'K476', 'R637'],
    ['NEW52689', 'A089', 'V733', 'W158', 'A640', 'H331'],
    ['NEW67368', 'Z079', 'J617', 'G878', 'W111', 'Q500'],
    ['NEW72348', 'J547', 'B222', 'G492', 'R551', 'S490'],
    ['NEW34469', 'Y990', 'H898', 'W673', 'L967', 'M829'],
]

# Create DataFrames
columns = ['id', 'col1', 'col2', 'col3', 'col4', 'col5']
df1 = pd.DataFrame(data1, columns=columns)
df2 = pd.DataFrame(data2, columns=columns)
expected = {'gt': 5, 'tp': 4, 'fp': 2, 'fn': 1}

evaluator = DatasetEvaluator(df1, df2, expected, threshold=3, match_column='id')
evaluator.preproccess()
evaluator.evaluate()
evaluator.calculateStatistics()
evaluator.printResults()

preproccess took 0.0026 seconds
 ID00005
 ID00009
 ID00007
 ID00004
 ID10004
 NEW80187
 NEW30110
 NEW72832
 NEW30110
 NEW81243
 NEW52689
 NEW67368
 NEW72348
 NEW34469
evaluate took 0.0003 seconds
calculateStatistics took 0.0000 seconds
Expected: {'gt': 5, 'tp': 4, 'fp': 2, 'fn': 1}
Ground Truth Size: 5
True Positives: 1
False Positives: 0
False Negatives: 4
Precision: 1.0000
Recall: 0.2000


In [6]:
evaluator.hashed_bucket

defaultdict(list,
            {'': ['ID00005',
              'ID00009',
              'ID00007',
              'ID00004',
              'ID10004',
              'NEW80187',
              'NEW30110',
              'NEW72832',
              'NEW30110',
              'NEW81243',
              'NEW52689',
              'NEW67368',
              'NEW72348',
              'NEW34469']})

In [11]:
time1 =  578.75
time2 = 114.03

print(f"Pipeline 1 avg time: {time1} sec")
print(f"Pipeline 2 avg time: { time2} sec")

if time1 < time2:
    print(f"{(1 - time1/time2)*100:.2f}% increase time ---> not improvement")
else:
    print(f"-{(1 - time2/time1)*100:.2f}% reduce time ---> improvement")

Pipeline 1 avg time: 578.75 sec
Pipeline 2 avg time: 114.03 sec
-80.30% reduce time ---> improvement


In [ ]:
1250 Elapsed Time: 1.72 seconds
2500 Elapsed Time: 7.02 seconds 
5000 Elapsed Time: 28.09 seconds
10000 Elapsed Time: 114.03 seconds


1250 Elapsed Time: 6.29 seconds
2500 Elapsed Time: 26 seconds 
5000 Elapsed Time: 108.56 seconds
10000 Elapsed Time: 578.75 seconds

In [2]:
dataset = SyntheticMatcherDataset(size=100 ,  ground_truth_ratio=0.25, datasets_ratio = (1, 2), true_positive_ratio=0.75, threshold=3)
df1, df2 = dataset.df1, dataset.df2
expected = dataset.expected

evaluator = DatasetEvaluator(df1, df2, expected, threshold=3, trim=1, match_column="id")
evaluator.preproccess()
evaluator.evaluate()
evaluator.calculateStatistics()
evaluator.printResults()

preproccess took 0.0433 seconds
evaluate took 0.0361 seconds
calculateStatistics took 0.0001 seconds
Expected: {'gt': 25, 'tp': 18, 'fp': 13, 'fn': 7}
Ground Truth Size: 25
True Positives: 4
False Positives: 7
False Negatives: 21
Precision: 0.3636
Recall: 0.1600


In [7]:
evaluator.chunked_df2

{'P56S21B71K97Q10': array(['P56', '21B', '1K9'], dtype='<U3'),
 'T01B47V34T28G63': array(['T01', '47V', '4T2'], dtype='<U3'),
 'V51S24M90P91M51': array(['V51', '24M', '0P9'], dtype='<U3'),
 'J12H49P90D55E62': array(['J12', '49P', '0D5'], dtype='<U3'),
 'Y06W79K39Y74R43': array(['Y06', '79K', '9Y7'], dtype='<U3'),
 'D52L02N28R84X43': array(['D52', '02N', '8R8'], dtype='<U3'),
 'T47V06F91O14H64': array(['T47', '06F', '1O1'], dtype='<U3'),
 'L57M10W18K85H13': array(['L57', '10W', '8K8'], dtype='<U3'),
 'Y93P98Z46J62V55': array(['Y93', '98Z', '6J6'], dtype='<U3'),
 'V71W78A02L84I87': array(['V71', '78A', '2L8'], dtype='<U3'),
 'M97H97U84L99K33': array(['M97', '97U', '4L9'], dtype='<U3'),
 'P67A50P70L65K50': array(['P67', '50P', '0L6'], dtype='<U3'),
 'Q21D85S70Y14U53': array(['Q21', '85S', '0Y1'], dtype='<U3'),
 'K57Y41L11R97O02': array(['K57', '41L', '1R9'], dtype='<U3'),
 'A89R91F76W35V22': array(['A89', '91F', '6W3'], dtype='<U3'),
 'K27P57A54R42R48': array(['K27', '57A', '4R4'], dtype=

In [ ]:
preproccess took 0.6206 seconds
evaluate took 24.9171 seconds
calculateStatistics took 1.9571 seconds
Expected: {'gt': 625, 'tp': 468, 'fp': 351, 'fn': 157}
Ground Truth Size: 625
True Positives: 468
False Positives: 351
False Negatives: 157
Precision: 0.5714
Recall: 0.7488

preproccess took 0.7642 seconds
evaluate took 21.1607 seconds
calculateStatistics took 2.9809 seconds
Expected: {'gt': 625, 'tp': 468, 'fp': 351, 'fn': 157}
Ground Truth Size: 625
True Positives: 468
False Positives: 351
False Negatives: 157
Precision: 0.5714
Recall: 0.7488

In [121]:
def fast_chunk(s):
    bs = s.encode('utf-8')
    chunk_len = len(bs) // 4 * 4  # ensure multiple of 4
    return np.frombuffer(bs[:chunk_len], dtype='S4')
    
# Step 1: Find best 3-column combination (least unique combinations)
columns = ['col1', 'col2', 'col3', 'col4', 'col5']
unique_counts = []

for cols in combinations(columns, 3):
    count = df2[list(cols)].astype(str).agg(''.join, axis=1).nunique()
    unique_counts.append((cols, count))

best_combination = min(unique_counts, key=lambda x: x[1])
best_cols = list(best_combination[0])

# Step 2: Build index based on best 3 columns
df2['index_key'] = df2[best_cols].astype(str).agg(''.join, axis=1)
df2_proc = df2.apply(lambda x: (x['id'], ''.join(map(str, x[1:6])), x['index_key']), axis=1).to_numpy()

df1['index_key'] = df1[best_cols].astype(str).agg(''.join, axis=1)
df1_proc = df1.apply(lambda x: (x['id'], ''.join(map(str, x[1:6])), x['index_key']), axis=1).to_numpy()


# Step 3: Build hashed buckets from df2 (by index_key)
hashed_bucket = defaultdict(lambda: defaultdict(list))

for id_val, full_str, index_key in df2_proc:
    hashed_bucket[index_key][full_str].append(id_val)

# Precompute chunked df1 strings once
chunked_df1 = [(hash, fast_chunk(combined), match_id) for match_id, combined, hash in df1_proc]

start = time.time()

for index_key, df1_chunks, match_id in chunked_df1:
    
    if index_key not in hashed_bucket:
       continue
        
    for full_str in hashed_bucket[key]:
        df2_chunks = fast_chunk(full_str)
        if len(df1_chunks) != len(df2_chunks):
            continue
        match_count = np.count_nonzero(df1_chunks == df2_chunks)
        if match_count >= 3:
            hashed_bucket[key][full_str].append(match_id)
            break

print(time.time() - start)

flat = []
for subdict in hashed_bucket.values():
    for ids in subdict.values():
        flat.append(ids)

fp, tp, fn = 0, 0, 0

for bucket in flat:
    if len(bucket) > 1:
        ids_to_check = np.array(bucket[1:])
        if (any(np.isin(ids_to_check, ground_truth_ids_np))):
            tp += 1
        else:
            fp += 1

fn = len(ground_truth_ids_np) - tp
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0

tp, fn, fp, precision, recall

0.0003428459167480469


(0, 125, 0, 0, 0.0)

In [118]:
hashed_bucket

defaultdict(<function __main__.<lambda>()>,
            {'B278M489W852': defaultdict(list,
                         {'B278M489W852O906B652': ['NEW82540']}),
             'Q555M940Z833': defaultdict(list,
                         {'Q555M940Z833O380B916': ['ID00009']}),
             'D770C000G960': defaultdict(list,
                         {'D770C000G960Z097G376': ['NEW67896']}),
             'C678Z545I904': defaultdict(list,
                         {'C678Z545I904J243P353': ['ID00005']}),
             'F357C719A834': defaultdict(list,
                         {'F357C719A834T346B769': ['NEW28633']}),
             'K011A722I355': defaultdict(list,
                         {'K011A722I355H817C344': ['NEW54635']}),
             'S276Y283L554': defaultdict(list,
                         {'S276Y283L554P404F209': ['NEW91955']}),
             'G471R284O013': defaultdict(list,
                         {'G471R284O013D713C034': ['NEW22056']}),
             'V435W342L052': defaultdict(list,
       

In [102]:
from itertools import combinations
from collections import defaultdict
import numpy as np
import time

def fast_chunk(s):
    bs = s.encode('utf-8')
    chunk_len = len(bs) // 4 * 4  # ensure multiple of 4
    return np.frombuffer(bs[:chunk_len], dtype='S4')

ground_truth_ids_np = np.intersect1d(df1['id'], df2['id'])
# Step 1: Find best 3-column combination (least unique combinations)
columns = ['col1', 'col2', 'col3', 'col4', 'col5']
unique_counts = []

for cols in combinations(columns, 3):
    count = df2[list(cols)].astype(str).agg(''.join, axis=1).nunique()
    unique_counts.append((cols, count))

best_combination = min(unique_counts, key=lambda x: x[1])
best_cols = list(best_combination[0])

# Step 2: Build index based on best 3 columns
df2['index_key'] = df2[best_cols].astype(str).agg(''.join, axis=1)
df2_proc = df2.apply(lambda x: (x['id'], ''.join(map(str, x[1:6])), x['index_key']), axis=1).to_numpy()

df1['index_key'] = df1[best_cols].astype(str).agg(''.join, axis=1)
df1_proc = df1.apply(lambda x: (x['id'], ''.join(map(str, x[1:6])), x['index_key']), axis=1).to_numpy()



# Step 3: Build hashed buckets from df2 (by index_key)
hashed_bucket = defaultdict(lambda: defaultdict(list))

for id_val, full_str, index_key in df2_proc:
    hashed_bucket[index_key][full_str].append(id_val)

# Precompute chunked df1 strings once
chunked_df1 = [(hash, fast_chunk(combined), match_id) for match_id, combined, hash in df1_proc]

fp, fn, tp = 0, 0 , 0
start = time.time()

for index_key, df1_chunks, match_id in chunked_df1:
    for key in list(hashed_bucket.keys()):
        fast_key1 = fast_chunk(index_key)
        if np.count_nonzero(fast_key1 == fast_chunk(key)) == 0:
            continue

        for full_str in hashed_bucket[key]:
            df2_chunks = fast_chunk(full_str)
            if len(df1_chunks) != len(df2_chunks):
                continue

            match_count = np.count_nonzero(df1_chunks == df2_chunks)
            if match_count >= 3:
                hashed_bucket[key][full_str].append(match_id)
                break

print(time.time() - start)

flat = []
for subdict in hashed_bucket.values():
    for ids in subdict.values():
        flat.append(ids)

for bucket in flat:
    if len(bucket) > 1:
        ids_to_check = np.array(bucket[1:])
        mask = np.isin(ids_to_check, ground_truth_ids_np)
        matched |= set(ids_to_check[mask])
        fp += mask.size - np.count_nonzero(mask)

tp = len(matched)
fn = len(ground_truth_ids_np) - tp
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0

tp, fn, fp, precision, recall

0.9082980155944824


(2327, -2202, 69, 0.9712020033388982, 18.616)

In [2]:
import timeit

def pipeline1():
    dataset = SyntheticMatcherDataset(size=1000 , true_positive_ratio=0.70, threshold=3)
    df1, df2 = dataset.df1, dataset.df2
    expected = dataset.expected
    
    unique_tokens = pd.unique(pd.concat([df1.iloc[:, 1:6], df2.iloc[:, 1:6]], axis=0).stack())
    token_map = {token: idx for idx, token in enumerate(unique_tokens)}
    
    def map_row(row):
        return [token_map[val] for val in row[1:6]]
        
    # Keep column 0 as-is
    df1_ids = df1.iloc[:, [0]]
    
    # Apply mapping only on columns 1 to 5
    df1_mapped = df1.iloc[:, 1:6].apply(map_row, axis=1, result_type='expand')
    
    # Concatenate back with column 0
    df1_final = pd.concat([df1_ids, df1_mapped], axis=1)
    
    # Repeat for df2
    df2_ids = df2.iloc[:, [0]]
    df2_mapped = df2.iloc[:, 1:6].apply(map_row, axis=1, result_type='expand')
    df2_final = pd.concat([df2_ids, df2_mapped], axis=1)
        
    evaluator = DatasetEvaluator(df1, df2, expected, threshold=3, match_column="id")
    evaluator.evaluate()
    # evaluator.printResults()

def pipeline2():
    dataset = SyntheticMatcherDataset(size=1000 , true_positive_ratio=0.70, threshold=3)
    df1, df2 = dataset.df1, dataset.df2
    expected = dataset.expected
    
    evaluator = DatasetEvaluator(df1, df2, expected, threshold=3, match_column="id")
    evaluator.evaluate()
    # evaluator.printResults()

time1 = timeit.timeit(pipeline1, number=10)
time2 = timeit.timeit(pipeline2, number=10)

print(f"Pipeline 1 avg time: {time1 / 10:.4f} sec")
print(f"Pipeline 2 avg time: {time2 / 10:.4f} sec")

if time1 < time2:
    print(f"{1 - time1/time2:.2f}% increase time ---> not improvement")
else:
    print(f"-{1 - time2/time1:.2f}% reduce time ---> improvement")
    

Pipeline 1 avg time: 6.8511 sec
Pipeline 2 avg time: 5.4194 sec
-0.21% reduce time ---> improvement


In [7]:
Pipeline 1 avg time: 6.2479 sec
Pipeline 2 avg time: 6.0408 sec

Expected: {'gt': 125, 'tp': 87, 'fp': 38, 'fn': 38}
Ground Truth Size: 125
True Positives: 87
False Positives: 38
False Negatives: 38
Precision: 0.6960
Recall: 0.6960
Elapsed Time: 1.26 seconds

1746562494.410388

In [38]:
if time1 < time2:
    print(f"{1 - time1/time2:.2f}% increase time ---> not improvement")
else:
    print(f"-{1 - time2/time1:.2f}% reduce time ---> improvement")
    

-0.03% reduce time ---> improvement


0.9668528625618208

In [3]:
{'R746E298Q412L291R232': ['ID00005'], 'R822W179H017P323F298': ['ID00009'], 'Z011X716M948W967S702': ['ID00007'], 'N002E396N843V935S719': ['ID00004'], 'N002E396N553I453S459': ['ID10004']}

{'R746E298Q412L291R232': ['ID00005'],
 'R822W179H017P323F298': ['ID00009'],
 'Z011X716M948W967S702': ['ID00007'],
 'N002E396N843V935S719': ['ID00004'],
 'N002E396N553I453S459': ['ID10004']}

In [45]:
from itertools import combinations

columns = ['col1', 'col2', 'col3', 'col4', 'col5']

# Store result as (column_triplet, unique_count)
unique_counts = []

for cols in combinations(columns, 3):
    count = df2[list(cols)].agg(''.join, axis=1).nunique()
    unique_counts.append((cols, count))

# Find the combination with the minimum unique count
best_combination = min(unique_counts, key=lambda x: x[1])

print("Best column triplet (least unique values):", best_combination[0])
print("Number of unique combinations:", best_combination[1])

Best column triplet (least unique values): ('col1', 'col2', 'col3')
Number of unique combinations: 10000


In [7]:
df1_proc = df1.apply(lambda x: (x[0], ''.join(map(str, x[1:]))), axis=1).to_numpy()
df2_proc = df2.apply(lambda x: (x[0], ''.join(map(str, x[1:]))), axis=1).to_numpy()

In [41]:
asd1 = df2[['col1', 'col2', 'col3']].agg(''.join, axis=1).nunique(), ['col1', 'col2', 'col3']
asd2 = df2[['col2', 'col3', 'col4']].agg(''.join, axis=1).nunique(), ['col2', 'col3', 'col4']
asd3 = df2[['col3', 'col4', 'col5']].agg(''.join, axis=1).nunique(), ['col3', 'col4', 'col5']
min([asd1,asd2,asd3])

(100, ['col1', 'col2', 'col3'])

In [3]:
def fast_chunk(s):
    bs = s.encode('utf-8')
    chunk_len = len(bs) // 4 * 4  # ensure multiple of 4
    return np.frombuffer(bs[:chunk_len], dtype='S4')

array([b'F553', b'F553', b'F553', b'F553', b'F553'], dtype='|S4')

In [46]:
unique_counts

[(('col1', 'col2', 'col3'), 10000),
 (('col1', 'col2', 'col4'), 10000),
 (('col1', 'col2', 'col5'), 10000),
 (('col1', 'col3', 'col4'), 10000),
 (('col1', 'col3', 'col5'), 10000),
 (('col1', 'col4', 'col5'), 10000),
 (('col2', 'col3', 'col4'), 10000),
 (('col2', 'col3', 'col5'), 10000),
 (('col2', 'col4', 'col5'), 10000),
 (('col3', 'col4', 'col5'), 10000)]

In [ ]:
hashed_bucket

False

(4, 2, 3, 0.5714285714285714, 0.6666666666666666)

In [81]:
# Evaluate precision/recall
matched = set()
fp = 0
ground_truth_ids_np = np.intersect1d(df1['id'], df2['id'])

for bucket in hashed_bucket.values():
    if len(bucket) > 1:
        ids_to_check = np.array(bucket[1:])  # exclude the original
        mask = np.isin(ids_to_check, ground_truth_ids_np)
        matched |= set(ids_to_check[mask])
        fp += mask.size - np.count_nonzero(mask)

tp = len(matched)
fn = len(ground_truth_ids_np) - tp
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0

tp, fn, fp, precision, recall

(0, 6, 0, 0, 0.0)

In [80]:
np.array(list(np.intersect1d(df1['id'], df2['id'])))

array(['ID00001', 'ID00009', 'ID00011', 'ID00013', 'ID00014', 'ID00016'],
      dtype='<U7')

In [66]:
np.array(list(np.intersect1d(df1['id'], df2['id']))).size

6

In [73]:
ids_to_check

array(['ID00017'], dtype='<U7')

In [16]:
def fast_chunk(s: str, trim: int = 0):
    chunks = [s[i:i+4] for i in range(0, len(s) - len(s) % 4, 4)]
    if trim > 0:
        chunks = [chunk[:-trim] if len(chunk) > trim else '' for chunk in chunks]
    return np.array(chunks)

def combine_and_trim_chunks(row, trim: int = 0):
    chunks = [str(x) for x in row[1:]]  # skip ID
    trimmed_chunks = [c[:-1] if 1 > 0 else c for c in chunks]
    combined = ''.join(trimmed_chunks)
    return (row[0], combined)

In [4]:
fast_chunk("R746E298Q412L291R232", trim=1)

array(['R74', 'E29', 'Q41', 'L29', 'R23'], dtype='<U3')

In [11]:
combine_and_trim_chunks(["id","R746", "E298", "Q412", "L291","R232"], 1)

('id', 'R74E29Q41L29R23')

In [17]:
evaluator.df2.apply(combine_and_trim_chunks, axis=1)

0      (ID00005, R74E29Q41L29R23)
1      (ID00009, R82W17H01P32F29)
2      (ID00007, Z01X71M94W96S70)
3      (ID00004, N00E39N84V93S71)
4      (ID10004, N00E39N55I45S45)
5     (NEW80187, J54B22G49W67S49)
6     (NEW30110, N32K68T43C22W96)
7     (NEW72832, F87Q76H82Z15X67)
8     (NEW30110, R56C43M68Q68Q86)
9     (NEW81243, R76N68A10K47R63)
10    (NEW52689, A08V73W15A64H33)
11    (NEW67368, Z07J61G87W11Q50)
12    (NEW72348, J54B22G49R55S49)
13    (NEW34469, Y99H89W67L96M82)
dtype: object